##  MPI ping-pong
Normally, we would just run `mpirun -n 2 python 3pingpong.py`. In Jupyter notebooks, we need to start the [iparallel cluster](https://ipyparallel.readthedocs.io/en/latest/examples/Cluster%20API.html).

In [1]:
n = 2 # number of processes
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"] = "1" # OpenMPI flag to oversubscribe in case we have less cpus than n
from ipyparallel import Cluster
cluster = await Cluster(engines="mpi").start_and_connect(n=n, activate=True)

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

The cells need to be prefixed by `%%px`

In [2]:
%%px
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if size != 2:
    if rank == 0:
        print("This program requires exactly 2 processes")
    exit()

niter = 10000
message = 0
partner_rank = 1 - rank

print(f"Hello world from rank {rank} of {size}. My partner is {partner_rank}.")

[stdout:0] Hello world from rank 0 of 2. My partner is 1.


[stdout:1] Hello world from rank 1 of 2. My partner is 0.


The processes execute the code independently.
As a consequence, the second process can print the above string before the first process.
We synchronize all processes before timing

In [4]:
%%px
comm.Barrier()
start = MPI.Wtime()

for _ in range(niter):
    if rank == 0:
        comm.send(message, dest=partner_rank) # lowercase methods are used for python objects
        message = comm.recv(source=partner_rank)
    else:
        message = comm.recv(source=partner_rank)
        comm.send(message, dest=partner_rank)

end = MPI.Wtime()

if rank == 0:
    total_time = end - start # in seconds
    avg_time = (total_time / (2.0 * niter)) * 1e6  # one-way latency in microseconds
    print(f"Ping-pong latency: {avg_time:.3f} microseconds (avg over {niter} iterations)")

[stdout:0] Ping-pong latency: 3.828 microseconds (avg over 10000 iterations)
